# Vector Space Emdeddings of K Modal Logic Formulas

In [ ]:
import random
import numpy as np
from multiprocessing import Pool

Propositional letters (variables representing atomic propositions in logic)

In [ ]:
propositions = ['p', 'q', 'r', 's,', 't']

Logical and modal operators

In [ ]:
logical_operators = ['¬', '∧', '∨']  # '¬' (negation), '∧' (conjunction), '∨' (disjunction)
modal_operators = ['□', '◇']  # '□' (necessarily), '◇' (possibly)

Function to generate a random modal formula

In [ ]:
def generate_modal_formula(propositions, depth):
    """
    Recursively generates a random modal logic formula based on the given depth.

    Parameters:
    depth (int): The depth of the formula, controlling its complexity.

    Returns:
    str: A randomly generated modal formula as a string.
    """
    if depth == 0:
        # Base case: return a randomly chosen propositional letter
        return random.choice(propositions)
    else:
        # Randomly select an operator (either logical or modal)
        operator = random.choice(logical_operators + modal_operators)

        if operator in logical_operators:
            if operator == '¬':  # Unary logical operator (negation)
                return f'¬{generate_modal_formula(propositions, depth - 1)}'
            else:  # Binary logical operators (conjunction, disjunction)
                return f'({generate_modal_formula(propositions, depth - 1)} {operator} {generate_modal_formula(propositions, depth - 1)})'
        else:
            # Modal operators (□ and ◇) are unary
            return f'{operator}{generate_modal_formula(propositions, depth-1)}'


Function to generate a random Kripke model

In [ ]:
def generate_kripke_model(num_worlds, propositions):
    """
    Generates a random Kripke model with a given number of worlds, ensuring that no world is isolated.

    Parameters:
    num_worlds (int): The number of worlds in the Kripke model.
    propositions (list): A list of propositional variables.

    Returns:
    tuple: A tuple containing:
        - worlds (list): A list of world names (e.g., ['w0', 'w1', ...]).
        - relations (list of tuples): A randomly generated accessibility relation as pairs of worlds.
        - valuation (dict): A dictionary mapping each world to a truth assignment for propositional variables.
    """
    # Creating a list of world names (e.g., ['w0', 'w1', 'w2', ...])
    worlds = [f'w{i}' for i in range(num_worlds)]

    # Ensure that w0 is connected to all other worlds
    relations = []
    for i in range(1, num_worlds):
        relations.append((worlds[0], worlds[i]))  # Add an edge from w0 to every other world

    # Randomly add additional edges to ensure reachability
    for i in range(1, num_worlds):
        for j in range(num_worlds):
            if i != j and random.choice([True, False]):
                relations.append((worlds[i], worlds[j]))

    # Remove duplicate relations (if any)
    relations = list(set(relations))

    # Assigning a random truth value (True/False) to each propositional letter in each world
    valuation = {w: {p: random.choice([True, False]) for p in propositions} for w in worlds}

    return worlds, relations, valuation


Function to decompose a complex formula

In [ ]:
def decompose_formula(formula):
    """
    Decomposes a complex modal logic formula into its components:
    - The left operand
    - The main logical operator
    - The right operand

    Parameters:
    formula (str): A modal logic formula in string format.

    Returns:
    tuple: A tuple containing:
        - left_part (str or None): The left operand of the formula.
        - main_operator (str or None): The main logical operator.
        - right_part (str or None): The right operand of the formula.
        If the formula cannot be decomposed, returns (None, None, None).
    """
    # Remove outer parentheses if they enclose the entire formula
    formula = formula[1:-1]

    # Identify the main operator by tracking parenthesis depth
    depth = 0
    for i, char in enumerate(formula):
        if char == '(':
            depth += 1
        elif char == ')':
            depth -= 1
        elif char in logical_operators and depth == 0:
            # Main operator found at depth 0, split the formula at this point
            return formula[:i].strip(), char, formula[i+1:].strip()

    # If no valid decomposition is found, return None
    return None, None, None


Function for evaluating a formula on a world (i.e., model checking)

In [ ]:
def evaluate_formula(formula, world, worlds, relations, valuation):
    """
    Evaluates a modal logic formula starting specifically from world 'w1'.

    Parameters:
    formula (str): The modal logic formula to evaluate.
    world (str): (Ignored) Always set to 'w1'.
    worlds (list): List of all worlds in the Kripke model.
    relations (list of tuples): Accessibility relations.
    valuation (dict): Truth assignments for each world.

    Returns:
    bool: True if the formula is satisfied in world 'w1', False otherwise.
    """

    if formula in propositions:
        return valuation[world][formula]

    elif formula.startswith('¬'):
        return not evaluate_formula(formula[1:], world, worlds, relations, valuation)

    elif formula.startswith('('):
        subformula1, operator, subformula2 = decompose_formula(formula)

        if operator == '∧':
            return evaluate_formula(subformula1, world, worlds, relations, valuation) and \
                   evaluate_formula(subformula2, world, worlds, relations, valuation)

        elif operator == '∨':
            return evaluate_formula(subformula1, world, worlds, relations, valuation) or \
                   evaluate_formula(subformula2, world, worlds, relations, valuation)

    elif formula.startswith('□'):
        return all(evaluate_formula(formula[1:], w, worlds, relations, valuation)
                   for w in worlds if ('w0', w) in relations)

    elif formula.startswith('◇'):
        return any(evaluate_formula(formula[1:], w, worlds, relations, valuation)
                   for w in worlds if ('w0', w) in relations)

    return False


Function to iteratively enhance the satisfaction matrix

In [ ]:
def iterative_optimization(truth_matrix, formulas, models, depth=6, num_worlds_range=(2, 5), nthreads=1):
    """
    Iteratively ensures that all columns in the truth matrix are unique by adjusting formulas and models.
    The function continues until all columns are unique or the maximum number of iterations is reached.
    """
    pool = Pool(nthreads)
    n_formulas, m_models = truth_matrix.shape

    while True:
        # Check if all columns are unique
        unique_columns = len(set(map(tuple, truth_matrix.T))) == m_models

        if unique_columns:
            print("All columns are unique!")
            break  # Stop if all columns are unique

        # Attempt to generate a new valid model for each column
        for idx in range(m_models):
            num_worlds = random.randint(*num_worlds_range)
            new_model = generate_kripke_model(num_worlds, propositions)
            new_column_values = np.array([
                evaluate_formula(formulas[i], 'w0', new_model[0], new_model[1], new_model[2])
                for i in range(n_formulas)
            ], dtype=int)

            # Check if the new column is unique
            if not any(np.array_equal(new_column_values, truth_matrix[:, j]) for j in range(m_models)):
                models[idx] = new_model
                truth_matrix[:, idx] = new_column_values
                break  # Move on to the next column
        else:
            # If no unique column was found after trying all models, generate new formulas
            print("No unique model found, generating new formulas...")
            for i in range(n_formulas):
                # Generate a new modal formula with the same depth
                formulas[i] = generate_modal_formula(propositions, depth)

            # Re-evaluate all columns with the new formulas
            for idx in range(m_models):
                truth_matrix[:, idx] = np.array([
                    evaluate_formula(formulas[i], 'w0', models[idx][0], models[idx][1], models[idx][2])
                    for i in range(n_formulas)
                ], dtype=int)

    pool.close()
    pool.join()
    return truth_matrix, formulas, models


Function to generate the vector space embeddings

In [ ]:
def generate_embedding(dimension, formula_depth, universe_range, nthreads=1):
    """
    Generates an embedding matrix for modal logic formulas and Kripke models.
    Starts with a 3x3 truth matrix, formulas, and models.
    """
    # Initialization with a 3x3 matrix, formulas, and predefined models
    formulas = [generate_modal_formula(propositions, random.randint(1, formula_depth)) for _ in range(3)]
    models = [generate_kripke_model(random.randint(2, universe_range), propositions) for _ in range(3)]

    # Creation of the 3x3 truth_matrix
    truth_matrix = np.zeros((3, 3), dtype=int)
    for i in range(3):
        for j in range(3):
            worlds, relations, valuation = models[j]  # Access elements of the tuple
            truth_matrix[i, j] = evaluate_formula(formulas[i], 'w0', worlds, relations, valuation)

    print("Initial 3x3 matrix:\n", truth_matrix)
    print("Initial formulas:", formulas)
    print("Initial models:", models)

    pool = Pool(processes=nthreads)

    # Start from the 4th iteration, since we already have a 3x3 matrix
    for i in range(3, dimension):  # Remaining iterations to reach the desired dimension
        print(f"--- Iteration {i+1} ---")

        depth = random.randint(1, formula_depth)
        num_worlds = random.randint(2, universe_range)

        # Generate a new model and formula
        new_formula = generate_modal_formula(propositions, depth)
        new_worlds, new_relations, new_valuation = generate_kripke_model(num_worlds, propositions)

        print("New formula generated:", new_formula)
        print("New model generated with", num_worlds, "worlds.")

        # Add the new formula and new model to the lists
        formulas.append(new_formula)
        # We use a tuple for the new model
        models.append((new_worlds, new_relations, new_valuation))

        # Expand truth_matrix to include the new formula and the new model
        new_size = len(formulas)  # The new dimension equals the number of formulas/models
        expanded_matrix = np.zeros((new_size, new_size), dtype=int)

        # Copy the old matrix into the new one
        expanded_matrix[:-1, :-1] = truth_matrix

        # Evaluate the new formula on the existing models (if any)
        new_row = np.array([
            evaluate_formula(new_formula, 'w0', m[0], m[1], m[2])  # Access elements of the tuple
            for m in models
        ], dtype=int)

        # Evaluate the existing formulas on the new model (if any)
        new_column = np.array([
            evaluate_formula(f, 'w0', new_worlds, new_relations, new_valuation)
            for f in formulas
        ], dtype=int)

        # Add the new formula's value on the new model
        new_value = int(evaluate_formula(new_formula, 'w0', new_worlds, new_relations, new_valuation))

        # Insert values into the new matrix
        expanded_matrix[-1, :] = new_row      # New row
        expanded_matrix[:, -1] = new_column   # New column
        expanded_matrix[-1, -1] = new_value   # Diagonal value

        # Update the matrix to pass to the optimization
        truth_matrix = expanded_matrix

        print("Number of formulas:", len(formulas))
        print("Number of models:", len(models))
        print("Truth matrix dimensions:", truth_matrix.shape)

        # Pass the expanded matrix to the optimization
        truth_matrix, formulas, models = iterative_optimization(
            truth_matrix, formulas, models,
            num_worlds_range=(2, universe_range), nthreads=nthreads
        )

        # Check if the optimization has failed (if there are identical columns)
        if len(set(map(tuple, truth_matrix.T))) < truth_matrix.shape[1]:
            print("Optimization not successful")
            break

        # Print the updated matrix and move on to the next iteration
        print("Truth Matrix after optimization:\n", truth_matrix)

    pool.close()
    pool.join()
    return truth_matrix, formulas, models


Generate vector space embeddings

In [ ]:
truth_matrix, formulas, models = generate_embedding(6, 6, 6)

Results

In [ ]:
print(truth_matrix)
print(formulas)
print(models)